In [5]:
import pandas as pd
import numpy as np
import pingouin as pg
from pathlib import Path
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import os.path as op

In [10]:
ANNOT_DIR = Path("derivatives/annotations")
OUT_DIR = Path("derivatives/caps/interrater")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [13]:
annot_df = pd.read_csv(op.join(ANNOT_DIR, "S01E01R01.csv"))  

In [15]:
annot_df.head()

,index,valence_1,valence_2,arousal_1,arousal_2
0,S01E01R01_clip0000,5,5,3,1
1,S01E01R01_clip0001,5,5,3,1
2,S01E01R01_clip0002,5,5,3,1
3,S01E01R01_clip0003,5,5,3,1
4,S01E01R01_clip0004,5,5,3,1


In [16]:
# Collect columns with 'valence' in their names
valence_cols = [col for col in annot_df.columns if 'valence' in col.lower()]
print("Valence columns found:")
print(valence_cols)
print(f"\nNumber of valence columns: {len(valence_cols)}")

# Display the valence data
print("\nValence data preview:")
annot_df[valence_cols].head()

Valence columns found:
['valence_1', 'valence_2']

Number of valence columns: 2

Valence data preview:


,valence_1,valence_2
0,5,5
1,5,5
2,5,5
3,5,5
4,5,5


In [18]:
# Collect columns with 'valence' in their names
arousal_cols = [col for col in annot_df.columns if 'arousal' in col.lower()]
print("Arousal columns found:")
print(arousal_cols)
print(f"\nNumber of arousal columns: {len(arousal_cols)}")

# Display the arousal data
print("\nArousal data preview:")
annot_df[arousal_cols].head()

Arousal columns found:
['arousal_1', 'arousal_2']

Number of arousal columns: 2

Arousal data preview:


,arousal_1,arousal_2
0,3,1
1,3,1
2,3,1
3,3,1
4,3,1
